<a href="https://colab.research.google.com/github/WormBase/wormcells-notebooks/blob/main/wormcells_wrangle_packer2019_h5ad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook will create the `packer2019.h5ad` file 


**A lineage-resolved molecular atlas of C. elegans embryogenesis at single-cell resolution**

_Jonathan S. Packer, Qin Zhu, Chau Huynh, Priya Sivaramakrishnan, Elicia Preston, Hannah Dueck, Derek Stefanik, Kai Tan, Cole Trapnell, Junhyong Kim†,
Robert H. Waterston, John I. Murray_

Science  20 Sep 2019: Vol. 365, Issue 6459, eaax1971

[https://doi.org/10.1126/science.aax1971](https://doi.org/10.1126/science.aax1971)

### Data description and links

89,701 cells profiled with 10xv2 on multiple timepoints  of embryo development

Raw sequencing data is available at https://www.ncbi.nlm.nih.gov/bioproject/658829

Samples wrangled here:
```
GSM3618670	UW synchronized 300 min post bleach
GSM3618671	UW synchronized 400 min post bleach
GSM3618672	UW synchronized 500 min post bleach batch 1
GSM3618673	UW synchronized 500 min post bleach batch 2
GSM3618674	UPenn mixed embryo batch r17
GSM3618675	UPenn mixed embryo batch b01
GSM3618676	UPenn mixed embryo batch b02
```

We use the files provided on GEO and download them from the corresponding ftp link
```
# GSE126954_cell_annotation.csv.gz	2.7 Mb	
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_cell_annotation.csv.gz

# GSE126954_gene_annotation.csv.gz	173.1 Kb	
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_gene_annotation.csv.gz

# GSE126954_gene_by_cell_count_matrix.txt.gz	249.2 Mb	
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_gene_by_cell_count_matrix.txt.gz
```


## Data wrangling conventions

The convention of how WormBase wrangles single cell RNA sequencing data into the [anndata](https://anndata.readthedocs.io/en/stable/) format in `.h5ad` files with standard fields is described here: https://github.com/WormBase/anndata-wrangling 

As possible, we attempt to keep the field names lower case, short, descriptive, and only using valid Python variable names so they may be accessed via the syntax `adata.var.field_name` 

Our goal is to standartize the naming convention for frequently used fields so that code may be reused without headaches changing variable names. The .h5ad file should only contain genes and cells with at least one count.


In [1]:
!pip install anndata --quiet
import anndata 
import pandas as pd

anndata.__version__

     |████████████████████████████████| 133kB 4.0MB/s 


'0.7.6'

In [2]:
### download the author provided files  files from Caltech data

!wget -O GSE126954_cell_annotation.csv.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_cell_annotation.csv.gz
!wget -O GSE126954_gene_annotation.csv.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_gene_annotation.csv.gz
!wget -O GSE126954_gene_by_cell_count_matrix.txt.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_gene_by_cell_count_matrix.txt.gz

--2021-05-26 02:55:28--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_cell_annotation.csv.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2814304 (2.7M) [application/x-gzip]
Saving to: ‘GSE126954_cell_annotation.csv.gz’

GSE126954_cell_anno 100%[===================>]   2.68M  --.-KB/s    in 0.1s    

2021-05-26 02:55:28 (25.5 MB/s) - ‘GSE126954_cell_annotation.csv.gz’ saved [2814304/2814304]

--2021-05-26 02:55:28--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE126nnn/GSE126954/suppl/GSE126954_gene_annotation.csv.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.228, 165.112.9.229, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.228|:443... connected.
HTTP request sent, awaiting

In [3]:
### LOAD CELL ANNOTATION DATAFRAME
### wrangles names to conform to convention

cells = pd.read_csv('GSE126954_cell_annotation.csv.gz')
display(cells.head())
cells=cells.fillna('unlabeled')
cells['barcode']=cells.cell.str.split('-', expand=True)[0]
cells=cells.rename(columns={'batch':'sample_batch',
                            'cell.subtype':'cell_subtype',
                            'cell.type':'cell_type',
                            'raw.embryo.time':'raw_embryo_time',
                            'embryo.time':'embryo_time',
                            'time.point':'time_point'
                            })
cells.index=cells['sample_batch']+'+'+cells['barcode']
cells['study']='packer2019'
cells['sample']=cells['sample_batch']
cells['sample_description']=cells['sample'].map({
'Waterston_300_minutes':'GSM3618670    UW synchronized 300 min post bleach',
'Waterston_400_minutes':'GSM3618671    UW synchronized 400 min post bleach',
'Waterston_500_minutes_batch_1':'GSM3618672    UW synchronized 500 min post bleach batch 1',
'Waterston_500_minutes_batch_2':'GSM3618673    UW synchronized 500 min post bleach batch 2',
'Murray_r17':'GSM3618674    UPenn mixed embryo batch r17',
'Murray_b01':'GSM3618675    UPenn mixed embryo batch b01',
'Murray_b02':'GSM3618676    UPenn mixed embryo batch b02'    
})
cells=cells[['study','sample_batch','sample','cell_type','cell_subtype','sample_description','barcode','raw_embryo_time','embryo_time','time_point','lineage']]
display(cells.head())

### CHECK THAT CELLS DATAFRAME CONFORMS TO ADATA NAMING CONVENTION
### must contain at least columns `gene_id` and `gene_name`
for column_name in ['study','sample_batch','sample','sample_description','barcode','cell_type','cell_subtype']:
    if column_name not in cells.columns:
        raise ValueError(column_name + ' is not a name in gene dataframe columns')
         
print('cells column names look ok!')

,Unnamed: 0,cell,n.umi,time.point,batch,Size_Factor,cell.type,cell.subtype,plot.cell.type,raw.embryo.time,embryo.time,embryo.time.bin,raw.embryo.time.bin,lineage,passed_initial_QC_or_later_whitelisted
0,AAACCTGAGACAATAC-300.1.1,AAACCTGAGACAATAC-300.1.1,1630,300_minutes,Waterston_300_minutes,1.023195,Body_wall_muscle,BWM_head_row_1,BWM_head_row_1,360,380.0,330-390,330-390,MSxpappp,True
1,AAACCTGAGGGCTCTC-300.1.1,AAACCTGAGGGCTCTC-300.1.1,2319,300_minutes,Waterston_300_minutes,1.458210,NaN,NaN,NaN,260,220.0,210-270,210-270,MSxapaap,True
2,AAACCTGAGTGCGTGA-300.1.1,AAACCTGAGTGCGTGA-300.1.1,3719,300_minutes,Waterston_300_minutes,2.338283,NaN,NaN,NaN,270,230.0,210-270,270-330,NaN,True
3,AAACCTGAGTTGAGTA-300.1.1,AAACCTGAGTTGAGTA-300.1.1,4251,300_minutes,Waterston_300_minutes,2.659051,Body_wall_muscle,BWM_anterior,BWM_anterior,260,280.0,270-330,210-270,Dxap,True
4,AAACCTGCAAGACGTG-300.1.1,AAACCTGCAAGACGTG-300.1.1,1003,300_minutes,Waterston_300_minutes,0.629610,Ciliated_amphid_neuron,AFD,AFD,350,350.0,330-390,330-390,ABalpppapav/ABpraaaapav,True


,study,sample_batch,sample,cell_type,cell_subtype,sample_description,barcode,raw_embryo_time,embryo_time,time_point,lineage
Waterston_300_minutes+AAACCTGAGACAATAC,packer2019,Waterston_300_minutes,Waterston_300_minutes,Body_wall_muscle,BWM_head_row_1,GSM3618670 UW synchronized 300 min post bleach,AAACCTGAGACAATAC,360,380.0,300_minutes,MSxpappp
Waterston_300_minutes+AAACCTGAGGGCTCTC,packer2019,Waterston_300_minutes,Waterston_300_minutes,unlabeled,unlabeled,GSM3618670 UW synchronized 300 min post bleach,AAACCTGAGGGCTCTC,260,220.0,300_minutes,MSxapaap
Waterston_300_minutes+AAACCTGAGTGCGTGA,packer2019,Waterston_300_minutes,Waterston_300_minutes,unlabeled,unlabeled,GSM3618670 UW synchronized 300 min post bleach,AAACCTGAGTGCGTGA,270,230.0,300_minutes,unlabeled
Waterston_300_minutes+AAACCTGAGTTGAGTA,packer2019,Waterston_300_minutes,Waterston_300_minutes,Body_wall_muscle,BWM_anterior,GSM3618670 UW synchronized 300 min post bleach,AAACCTGAGTTGAGTA,260,280.0,300_minutes,Dxap
Waterston_300_minutes+AAACCTGCAAGACGTG,packer2019,Waterston_300_minutes,Waterston_300_minutes,Ciliated_amphid_neuron,AFD,GSM3618670 UW synchronized 300 min post bleach,AAACCTGCAAGACGTG,350,350.0,300_minutes,ABalpppapav/ABpraaaapav


cells column names look ok!


In [4]:
genes = pd.read_csv('GSE126954_gene_annotation.csv.gz', index_col=0)
display(genes.head())

,id,gene_short_name
WBGene00010957,WBGene00010957,nduo-6
WBGene00010958,WBGene00010958,ndfl-4
WBGene00010959,WBGene00010959,nduo-1
WBGene00010960,WBGene00010960,atp-6
WBGene00010961,WBGene00010961,nduo-2


In [5]:
### LOAD GENE DATAFRAME AND CHANGES NAMES TO CONVENTION
genes = pd.read_csv('GSE126954_gene_annotation.csv.gz', index_col=0)
display(genes.head())
genes=genes.rename(columns={'id':'gene_id','gene_short_name':'gene_name'})
display(genes)

### CHECK THAT GENES DATAFRAME CONFORMS TO ADATA NAMING CONVENTION
### must contain at least columns `gene_id` and `gene_name`
for column_name in ['gene_id','gene_name']:
    if column_name not in genes.columns:
        raise ValueError(column_name + ' is not a name in gene dataframe columns')
         
print('gene column names look ok!')

,id,gene_short_name
WBGene00010957,WBGene00010957,nduo-6
WBGene00010958,WBGene00010958,ndfl-4
WBGene00010959,WBGene00010959,nduo-1
WBGene00010960,WBGene00010960,atp-6
WBGene00010961,WBGene00010961,nduo-2


,gene_id,gene_name
WBGene00010957,WBGene00010957,nduo-6
WBGene00010958,WBGene00010958,ndfl-4
WBGene00010959,WBGene00010959,nduo-1
WBGene00010960,WBGene00010960,atp-6
WBGene00010961,WBGene00010961,nduo-2
...,...,...
WBGene00021597,WBGene00021597,spsb-1
WBGene00021596,WBGene00021596,spsb-2
WBGene00021595,WBGene00021595,Y46E12BL.2
WBGene00021594,WBGene00021594,tig-3


gene column names look ok!


In [12]:
adata=anndata.read_mtx('GSE126954_gene_by_cell_count_matrix.txt.gz')
adata=adata.T
adata.var=genes
adata.obs=cells

### CHECK THAT OBS AND VAR CONFORMS TO ADATA NAMING CONVENTION
for column_name in ['study','sample_batch','sample','sample_description','barcode','cell_type','cell_subtype']:
    if column_name not in adata.obs.columns:
        raise ValueError(column_name + ' is not a name in gene adata.var columns')
for column_name in ['gene_id','gene_name']:
    if column_name not in adata.var.columns:
        raise ValueError(column_name + ' is not a name in gene adata.obs columns')
         
print('adata.var and adata.obs column names look ok!')
adata.write_h5ad('packer2019.h5ad')
adata

... storing 'study' as categorical
... storing 'sample_batch' as categorical
... storing 'sample' as categorical
... storing 'cell_type' as categorical
... storing 'cell_subtype' as categorical
... storing 'sample_description' as categorical


adata.var and adata.obs column names look ok!


... storing 'barcode' as categorical
... storing 'time_point' as categorical
... storing 'lineage' as categorical


AnnData object with n_obs × n_vars = 89701 × 20222
    obs: 'study', 'sample_batch', 'sample', 'cell_type', 'cell_subtype', 'sample_description', 'barcode', 'raw_embryo_time', 'embryo_time', 'time_point', 'lineage'
    var: 'gene_id', 'gene_name'